In [7]:
# Imports
import pandas as pd 
import numpy as np 
import cv2
import os
import pywt
import shutil
import seaborn as sb
import matplotlib.pyplot as plt 
%matplotlib inline


SyntaxError: invalid character in identifier (<ipython-input-7-3f75c9656201>, line 2)

In [2]:
# Examining a sample Image 

test_img = cv2.imread('../Datasets/lionel_messi/_111066400_messi.jpg')
test_img.shape


SyntaxError: invalid character in identifier (<ipython-input-2-2f947c8480cb>, line 3)

In [ ]:

# Lets see the image without the RGB Values 

gray_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
gray_img.shape

plt.imshow(gray_img, cmap='gray')


# Data Cleaning

In [ ]:
"""

To clean the data i need to :

1. Crop Images.
2. Check if an image has a FACE and TWO EYES (Using Harcascade)
3. Check if the image is of the celebrity i want.
4. Check for noise in the images



"""

### Exploring Diffrent Cascade Methods 

In [ ]:
# Initialising Cascade Models

face_cascader = cv2.CascadeClassifier('../OpenCV_Dependencies/haarcascades/haarcascade_frontalface_default.xml')
eye_cascader = cv2.CascadeClassifier('../OpenCV_Dependencies/haarcascades/haarcascade_eye.xml')

In [ ]:
# Applying Face Cascade Model to Image

face_detector = face_cascader.detectMultiScale(gray_img, 1.3, 5)
face_detector

In [ ]:
# The 4 Values represent : X, Y , Widht and Height

In [ ]:
"""
# Plotting the Detected Face 

(x_cord, y_cord, width, height) = face_detector[0]

face_detected_img = cv2.rectangle(test_img, (x_cord, y_cord), (x_cord+width, y_cord+height), (0, 255, 0), 2)
plt.imshow(face_detected_img) """



"""

# Applying Eye Cascade + Face Cascade Model to Image

cv2.destroyAllWindows()

for ((x_cord, y_cord, width, height)) in face_detector :

    face_detected_img = cv2.rectangle(test_img, (x_cord, y_cord), (x_cord+width, y_cord+height), (0, 255, 0), 2)

    roi_gray = gray_img[y_cord:y_cord+height, x_cord:x_cord+width]
    roi_color = face_detected_img[y_cord:y_cord+height, x_cord:x_cord+width]

    eye_detector = eye_cascader.detectMultiScale(roi_gray)

    for ((eye_x_cord, eye_y_cord, eye_width, eye_height)) in eye_detector :


        cv2.rectangle(roi_color, (eye_x_cord, eye_y_cord), (eye_x_cord+eye_width, eye_y_cord+eye_height), (0, 255, 0), 2)





plt.figure()
plt.imshow(face_detected_img, cmap='gray')
plt.show()





"""

# Cropped Image with Face and Eyes Detected
plt.imshow(roi_color) 

In [ ]:



# Lets Create a Function so that we can apply later

def image_detector_cropper(path_to_image) :

        image = cv2.imread(path_to_image)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        face_detector = face_cascader.detectMultiScale(gray_image, 1.3, 5)
        
        
        for ((x_cord, y_cord, width, height)) in face_detector :

                roi_gray = gray_image[y_cord:y_cord+height, x_cord:x_cord+width]
                roi_color = image[y_cord:y_cord+height, x_cord:x_cord+width]
                eyes_detector = eye_cascader.detectMultiScale(roi_gray)

                if (len(eyes_detector) >= 2):

                    return roi_color


plt.imshow(image_detector_cropper('../Datasets/lionel_messi/_111066400_messi.jpg'))

In [ ]:
# Getting Individual Directories

path_to_datasets = "../Datasets/"
path_for_cropped_datasets = "../Cropped_Datasets/"

image_directories = [] # Will Hold Names of Directories

for directory in os.scandir(path_to_datasets) :

    if directory.is_dir() :

        image_directories.append(directory.path)

In [ ]:
# Saving to Cropped Datasets 

# Check if Directory Exists else Create
if os.path.exists(path_for_cropped_datasets) :

    shutil.rmtree(path_for_cropped_datasets)

os.mkdir(path_for_cropped_datasets)


# Create new individual directories

cropped_img_directories = []
sports_celeb_names = {}


for directory in image_directories :

    count = 1
    celeb_name = directory.split('/')[-1]

    for celeb_img in os.scandir(directory) :

        cropped_img = image_detector_cropper(celeb_img.path)

        if cropped_img is not None :

            # Saved to Cropped Folder

            cropped_img_folder = path_for_cropped_datasets + celeb_name

            if not os.path.exists(cropped_img_folder) :

                os.makedirs(cropped_img_folder)
                cropped_img_directories.append(cropped_img_folder)


                print("Generating Cropped Image Directory for : ", cropped_img_folder)

            # Saving Cropped Images
            croppped_filename = celeb_name + str(count) + ".png"
            cropped_filepath = cropped_img_folder + "/" + croppped_filename
            cv2.imwrite(cropped_filepath, cropped_img)
            sports_celeb_names.setdefault(celeb_name, [])
            sports_celeb_names[celeb_name].append(cropped_filepath)

            count  += 1

# Feature Engineering Using WaveletTranform


In [ ]:
# Function to Perfrom Wavelet Tranformation
def wavelet_transform(img, mode='haar', level=1):

    img_arr = img

    # Tranfrom Image 
    img_arr = cv2.cvtColor(img_arr, cv2.COLOR_RGB2GRAY)
    img_arr = np.float32(img_arr)
    img_arr /= 255

    # Compute Coefficients
    coefficients = pywt.wavedec2(img_arr, mode, level=level)
    coefficients_H = list(coefficients)
    coefficients_H[0] *= 0

    # Reconstruct Image
    img_arr_H = pywt.waverec2(coefficients_H, mode)
    img_arr_H *= 255
    img_arr_H = np.uint8(img_arr_H)


    return img_arr_H 


In [ ]:
img_harred = wavelet_transform(cropped_img, 'db1', 5)
plt.imshow(img_harred, cmap='gray')



#### Stacking Raw and Tranformed Images Vertically for Comparison

In [ ]:
# Creating Numerical Classes for Celebrities

classes = {}
count = 1
for celeb_name in sports_celeb_names.keys():

    classes[celeb_name] = count
    count +=1
    
classes

In [ ]:

X = []
y =[]

for celeb_name, img_files in sports_celeb_names.items():

    for selected_img in img_files:

        img = cv2.imread(selected_img)

        if img is None:
            continue

        scaled_raw_img = cv2.resize(img, (32, 32))

        har_img = wavelet_transform(img, 'db1', 5)
        scaled_har_img = cv2.resize(har_img, (32, 32))

        stacked_img = np.vstack((scaled_raw_img.reshape(32*32*3, 1), scaled_har_img.reshape(32*32, 1)))

        X.append(stacked_img)
        y.append(celeb_name)

In [ ]:
X = np.array(X).reshape(len(X), 4096).astype(float)

# Machine Learning

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix



In [ ]:
my_pipeline = Pipeline([('scaler', StandardScaler()), ('svc', SVC(kernel= 'rbf', C=10))])
my_pipeline.fit(X_train, y_train)
print("Pipeline Accuracy = {accuracy}".format(accuracy=(my_pipeline.score(X_test, y_test)*100)))

In [ ]:
print(classification_report(y_test, my_pipeline.predict(X_test)))

## Implementing Grid Search 

In [ ]:
# Initialising Different Models

model_params = {

    'svm' : {
        'model' : svm.SVC(gamma='auto', probability=True),
        'params' : {
            'svc__C' :[1, 10, 100, 1000],
            'svc__kernel' : ['rbf', 'linear']
        }
    }, 

    'random_forest' : {
        'model' : RandomForestClassifier(),
        'params' : {
            'randomforestclassifier__n_estimators' :[1, 5, 10]
        }
    },

    'logistic_regression' : {
        'model' : LogisticRegression(solver= 'liblinear', multi_class='auto'),
        'params' : {
            'logisticregression__C' : [1, 5, 10]
        }
    }


}

In [ ]:
# Testing Models

scores = []
best_estimators = {}

for algorithm, model_configs in model_params.items():

    pipe_line = make_pipeline(StandardScaler(), model_configs['model'])
    clf = GridSearchCV(pipe_line, model_configs['params'], cv=5, return_train_score=False)
    clf.fit(X_train, y_train)
    scores.append({
        'model': algorithm,
        'best_score' : clf.best_score_,
        'best_parameters' : clf.best_params_
    })

    best_estimators[algorithm] = clf.best_estimator_

In [ ]:
models_df = pd.DataFrame(scores, columns=['model', 'best_score', 'best_parameters'])


models_df 

In [ ]:
print(best_estimators)


In [ ]:
# Performing on Test Data

best_estimators['svm'].score(X_test, y_test)



best_estimators['random_forest'].score(X_test, y_test)



best_estimators['logistic_regression'].score(X_test, y_test)



# Plotting Confusion Matrix for Best Model Selection

best_classifier = best_estimators['svm']

conf_mat = confusion_matrix(y_test, best_classifier.predict(X_test))

plt.figure(figsize=(7,10))
sb.heatmap(conf_mat, annot=True)
plt.xlabel('Predicted')
plt.ylabel('Actual')


In [ ]:
""" best_classifier = best_estimators['logistic_regression']

conf_mat = confusion_matrix(y_test, best_classifier.predict(X_test))

plt.figure(figsize=(7,10))
sb.heatmap(conf_mat, annot=True)
plt.xlabel('Predicted')
plt.ylabel('Actual') """




### Saving the Model


# Save as a pickle file
import joblib

joblib.dump(best_classifier, 'model.pkl')


# Saving classes 

import json

with open("classes_dict.json", "w") as file :

    file.write(json.dumps(classes))